In [40]:
import numpy as np
from pypointmatcher import pointmatcher as pm, pointmatchersupport as pms

In [41]:
def parse_translation(p_translation: str, p_cloud_dim: int):
    parsed_translation = np.identity(p_cloud_dim + 1)

    p_translation = p_translation.replace(',', ' ')

    translation_values = np.fromiter(p_translation.split(' '), np.float)

    for i, v in enumerate(translation_values):
        parsed_translation[i, p_cloud_dim] = v

    return parsed_translation

In [42]:
def parse_rotation(p_rotation: str, p_cloud_dim: int):
    parsed_rotation = np.identity(p_cloud_dim + 1)

    p_rotation = p_rotation.replace(',', ' ')
    p_rotation = p_rotation.replace(';', ' ')

    rotation_matrix = np.fromiter(p_rotation.split(' '), np.float)

    for i, v in enumerate(rotation_matrix):
        parsed_rotation[i // p_cloud_dim, i % p_cloud_dim] = v

    return parsed_rotation

In [43]:
PM = pm.PointMatcher
DP = PM.DataPoints
Parameters = pms.Parametrizable.Parameters

In [44]:
output_base_directory = ""
output_base_file = "test"

In [45]:
is_3D = True

if is_3D:
    # Load 3D point clouds
    ref = DP(DP.load('../data/Hokuyo_0.csv'))
    data = DP(DP.load('../data/Hokuyo_1.csv'))
    test_base = "3D"
else:
    # Load 2D point clouds
    ref = DP(DP.load('../data/2D_twoBoxes.csv'))
    data = DP(DP.load('../data/2D_oneBox.csv'))
    test_base = "2D"

In [46]:
icp = PM.ICP()
params = Parameters()

In [47]:
pms.setLogger(PM.get().LoggerRegistrar.create("FileLogger"))

In [48]:
# Prepare reading filters
name = "MinDistDataPointsFilter"
params["minDist"] = "1.0"
minDist_read = PM.get().DataPointsFilterRegistrar.create(name, params)
params.clear()

name = "RandomSamplingDataPointsFilter"
params["prob"] = "0.5"
rand_read = PM.get().DataPointsFilterRegistrar.create(name, params)
params.clear()

In [49]:
# Prepare reference filters
name = "MinDistDataPointsFilter"
params["minDist"] = "1.0"
minDist_ref = PM.get().DataPointsFilterRegistrar.create(name, params)
params.clear()

name = "RandomSamplingDataPointsFilter"
params["prob"] = "0.5"
rand_ref = PM.get().DataPointsFilterRegistrar.create(name, params)
params.clear()

In [50]:
# Prepare matching function
name = "KDTreeMatcher"
params["knn"] = "1"
params["epsilon"] = "3.16"
kdtree = PM.get().MatcherRegistrar.create(name, params)
params.clear()

In [51]:
# Prepare outlier filters
name = "TrimmedDistOutlierFilter"
params["ratio"] = "0.75"
trim = PM.get().OutlierFilterRegistrar.create(name, params)
params.clear()

In [52]:
# Prepare error minimization
name = "PointToPointErrorMinimizer"
pointToPoint = PM.get().ErrorMinimizerRegistrar.create(name)
params.clear()


In [53]:
# Prepare transformation checker filters
name = "CounterTransformationChecker"
params["maxIterationCount"] = "40"
maxIter = PM.get().TransformationCheckerRegistrar.create(name, params)
params.clear()

name = "DifferentialTransformationChecker"
params["minDiffRotErr"] = "0.001"
params["minDiffTransErr"] = "0.01"
params["smoothLength"] = "4"
diff = PM.get().TransformationCheckerRegistrar.create(name, params)
params.clear()

In [54]:
# Prepare inspector
# Comment out to write vtk files per iteration
name = "VTKFileInspector"
params["dumpDataLinks"] = "1"
params["dumpReading"] = "1"
params["dumpReference"] = "1"
vtkInspect = PM.get().InspectorRegistrar.create(name, params)
params.clear()

In [55]:
# Prepare transformation
name = "RigidTransformation"
rigid_trans = PM.get().TransformationRegistrar.create(name)

In [56]:
# Build ICP solution
icp.readingDataPointsFilters.append(minDist_read)
icp.readingDataPointsFilters.append(rand_read)

icp.referenceDataPointsFilters.append(minDist_ref)
icp.referenceDataPointsFilters.append(rand_ref)

icp.matcher = kdtree

icp.outlierFilters.append(trim)

icp.errorMinimizer = pointToPoint

icp.transformationCheckers.append(maxIter)
icp.transformationCheckers.append(diff)
icp.inspector = vtkInspect
icp.transformations.append(rigid_trans)

In [57]:
# Compute the transformation to express data in ref
T = icp(data, ref)

In [58]:
# Transform data to express it in ref
data_out = DP(data)

icp.transformations.apply(data_out, T)

In [59]:
# Save files to see the results
ref.save(f"{output_base_directory + test_base}_{output_base_file}_ref.vtk")
data.save(f"{output_base_directory + test_base}_{output_base_file}_data_in.vtk")
data_out.save(f"{output_base_directory + test_base}_{output_base_file}_data_out.vtk")

print(f"{test_base} ICP transformation:\n{T}".replace("[", " ").replace("]", " "))

3D ICP transformation:
   0.99343145 -0.11436385 -0.00352762  0.592276   
   0.11437842  0.99342835  0.00413194 -0.01413563 
   0.00303251 -0.00450766  0.9999866   0.00341469 
   0.          0.          0.          1.          
